In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avishek-hotel/sentiments.csv
/kaggle/input/hotel-text-data/text.txt


In [2]:

# Some used imports
%matplotlib inline
import pandas as pd
import numpy as np
import os
import glob
import matplotlib as mpl

# Just making the plots look better
mpl.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['font.size'] = 12

In [3]:
import nltk
from nltk.tokenize import sent_tokenize

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open("/kaggle/input/hotel-text-data/text.txt")
data = fp.read()
sentences=sent_tokenize(data)

## Sentences to pandas dataframe

In [4]:
df = pd.DataFrame(sentences,columns=["sentences"])

In [5]:
df

,sentences
0,The premises are very peaceful and well mainta...
1,The apartment was spacious though basically eq...
2,We enjoyed the fine pool and good food.
3,But the hospitality was overwhelming!
4,These people are so friendly and helpful that ...
...,...
407957,Police were in parking lot as I was pulling in.
407958,People walking through parking lot heading bac...
407959,"We were nearly eaten by the bed bugs, more agr..."
407960,I would not reccomend this hole even to my wor...


## Categories

In [6]:
cleanliness= ["satisfactory", "ample", "hygienic", "proper", "ambience", "odour","dirty","clean","smell","cleanliness"]
service = ["desk", "check in", "check out", "reliable", "fast", "convenient","service","hospitality"]
location= ["railway", "view","station","airport","distance","far","close","train", "metro","transport","market","mall","surrounding","areas","highway","traffic","out","location"]
value= ["price", "amount", "rate", "cheap", "worth", "low", 'money', "economical", "reasonable", "fee", "expensive", "charge","value"]
room= ["bed","bunkbeds","toilet", "bathroom", "shower","dryer","fridge", "space", "spacious", "outdated", "noisy","room"]
food= ["drink","breakfast","spicy", "food","tasty","tea","coffee", "buffet", "bar", "restaurant","dinner","lunch","brunch","delicious"]
facility= ["pool","gym","wifi","spa","internet","wireless","broken","parking","ventilation","maintained","facility","lot","premises"]
staff= ["friendly","helpful","reliable","quick","good","polite","staff"]

In [7]:
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
df["sentences_clean"] = df["sentences"].apply(lambda x: clean_text(x))

In [8]:
df

,sentences,sentences_clean
0,The premises are very peaceful and well mainta...,the premises are very peaceful and well mainta...
1,The apartment was spacious though basically eq...,the apartment was spacious though basically eq...
2,We enjoyed the fine pool and good food.,we enjoyed the fine pool and good food
3,But the hospitality was overwhelming!,but the hospitality was overwhelming
4,These people are so friendly and helpful that ...,these people are so friendly and helpful that ...
...,...,...
407957,Police were in parking lot as I was pulling in.,police were in parking lot as was pulling in
407958,People walking through parking lot heading bac...,people walking through parking lot heading bac...
407959,"We were nearly eaten by the bed bugs, more agr...",we were nearly eaten by the bed bugs more agre...
407960,I would not reccomend this hole even to my wor...,would not reccomend this hole even to my worst...


In [9]:
df1=df[df["sentences_clean"]!= ""]

In [10]:
df1

,sentences,sentences_clean
0,The premises are very peaceful and well mainta...,the premises are very peaceful and well mainta...
1,The apartment was spacious though basically eq...,the apartment was spacious though basically eq...
2,We enjoyed the fine pool and good food.,we enjoyed the fine pool and good food
3,But the hospitality was overwhelming!,but the hospitality was overwhelming
4,These people are so friendly and helpful that ...,these people are so friendly and helpful that ...
...,...,...
407957,Police were in parking lot as I was pulling in.,police were in parking lot as was pulling in
407958,People walking through parking lot heading bac...,people walking through parking lot heading bac...
407959,"We were nearly eaten by the bed bugs, more agr...",we were nearly eaten by the bed bugs more agre...
407960,I would not reccomend this hole even to my wor...,would not reccomend this hole even to my worst...


In [11]:
import re
wordlist=[]
for i in df1["sentences_clean"]:
    wordlist.append(re.split('\s+',i))

In [12]:
df1["sentences_clean"][5]

'after small accident with child that needed stitching in the hospital the brought us there and picked us up again and arranged some snacks during the waiting'

In [13]:
np.array(wordlist)

array([list(['the', 'premises', 'are', 'very', 'peaceful', 'and', 'well', 'maintained']),
       list(['the', 'apartment', 'was', 'spacious', 'though', 'basically', 'equipped']),
       list(['we', 'enjoyed', 'the', 'fine', 'pool', 'and', 'good', 'food']),
       ...,
       list(['we', 'were', 'nearly', 'eaten', 'by', 'the', 'bed', 'bugs', 'more', 'agressive', 'than', 'denali', 'bears']),
       list(['would', 'not', 'reccomend', 'this', 'hole', 'even', 'to', 'my', 'worst', 'enemy']),
       list(['next', 'day', 'we', 'needed', 'medical', 'help'])],
      dtype=object)

In [14]:
a=[]
for i in range(len(wordlist)):
    a.append(len(wordlist[i]))

In [15]:
from statistics import mean 

In [16]:
def cosine(wordlist,category):
    t=[]
    for i in wordlist:
        l1 =[];l2 =[] 
        X_set = {w for w in i}
        Y_set = {w for w in category}
        # form a set containing keywords of both strings  
        rvector = X_set.union(Y_set)  
        for w in rvector: 
            if w in X_set: l1.append(1) # create a vector 
            else: l1.append(0) 
            if w in Y_set: l2.append(1) 
            else: l2.append(0) 
        c = 0
        # cosine formula  
        for i in range(len(rvector)): 
                c+= l1[i]*l2[i] 
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        t.append(cosine) 
    return t

clc=cosine(wordlist,cleanliness)
sec=cosine(wordlist,service)
loc=cosine(wordlist,location)
vac=cosine(wordlist,value)
roc=cosine(wordlist,room)
foc=cosine(wordlist,food)
facc=cosine(wordlist,facility)
stac=cosine(wordlist,staff)

In [17]:
fuzzyc=np.c_[clc,sec]
fuzzyc=np.c_[fuzzyc,loc]
fuzzyc=np.c_[fuzzyc,vac]
fuzzyc=np.c_[fuzzyc,roc]
fuzzyc=np.c_[fuzzyc,foc]
fuzzyc=np.c_[fuzzyc,facc]
fuzzyc=np.c_[fuzzyc,stac]

In [18]:
fuzzyc

array([[0.        , 0.        , 0.        , ..., 0.        , 0.19611614,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.09449112, 0.09805807,
        0.13363062],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [19]:
maxpos=[]
for i in fuzzyc:
    maximum = np.max(i)
    tu=np.where(i == maximum)
    maxpos.append(tu[0])

In [20]:
pos=[]
for i in maxpos:
    pos.append(i[0])

In [21]:
categories=[]
for i in pos:
    if i==0:
        categories.append("cleanliness")
    if i==1:
        categories.append("service")
    if i==2:
        categories.append("location")
    if i==3:
        categories.append("value")
    if i==4:
        categories.append("room")
    if i==5:
        categories.append("food")
    if i==6:
        categories.append("facility")
    if i==7:
        categories.append("staff")

In [22]:
df1["categories"]=categories

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df1

,sentences,sentences_clean,categories
0,The premises are very peaceful and well mainta...,the premises are very peaceful and well mainta...,facility
1,The apartment was spacious though basically eq...,the apartment was spacious though basically eq...,room
2,We enjoyed the fine pool and good food.,we enjoyed the fine pool and good food,staff
3,But the hospitality was overwhelming!,but the hospitality was overwhelming,service
4,These people are so friendly and helpful that ...,these people are so friendly and helpful that ...,staff
...,...,...,...
407957,Police were in parking lot as I was pulling in.,police were in parking lot as was pulling in,facility
407958,People walking through parking lot heading bac...,people walking through parking lot heading bac...,room
407959,"We were nearly eaten by the bed bugs, more agr...",we were nearly eaten by the bed bugs more agre...,room
407960,I would not reccomend this hole even to my wor...,would not reccomend this hole even to my worst...,cleanliness


In [24]:
df1.to_csv('reviewsentences.csv',index=False)